# File Description

This notebook contains the most contains the latest version of the time dependent BN modeling for Tarawa, Kiribati.

# File Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import itertools
import os
import json
from datetime import datetime, time, timedelta
import pysmile
import pysmile_license
import sys
import json
sys.path.append('/src/python_classes')
import rpy2
# os.environ['R_HOME'] = 'C:\ProgramData\Anaconda3\Lib\R'
# %load_ext rpy2.ipython
!jupyter nbextension enable --py --sys-prefix ipyleaflet
from ipywidgets import interact, interactive, fixed, interact_manual
from ipyleaflet import *
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation
import geojson
import folium
from folium.plugins import FloatImage as FloatImage
from colormap import rgb2hex
import rpy2
os.environ['R_HOME'] = '/lib/R'
%load_ext rpy2.ipython

from BNModel import BNModel

from preprocessing_all_points import *
from preprocessing_points_spatially_temporally import *
from compile_model_t import *

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


In [3]:
# ### set location of file storage
# folder = 'BN_antonio_data'
# try:
#     os.makedirs(folder)
# except FileExistsError:
#     pass

# Data Prep

## Preprocessing

In [4]:
# Import and preprocess data
df_lagoon_profiles,df_ocean_profiles,inundation_dict,winds_dict,waves_dict,tide_dict,sla_dict,time_dict = \
    loading_tarawa_data()

## Create Variable Dictionary

In [5]:
def initialise_model_dictionaries():
    #### Don't include spaces in bin names. if no discretisation, just leave out that key
    lagoon_model_dict = {
        'variables':{
            'wind_u':{
                'label':'Wind u vector (m/s)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'wind_v':{
                'label':r'Wind v vector (m/s)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'uniform',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'Hs_offshore':{
                'label':'Offshore wave height (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','HighMid','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'Tm_offshore':{
                'label':'Offshore wave period (s)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'Dir_offshore':{
                'label':r'Offshore wave direction (degrees)',
                'discretisation':{
                    'n_bins':8,
                    'strategy':'kmeans',
                    'bin_names':['NNE','ENE','ESE','SSE','SSW','WSW','WNW','NNW']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'TWL':{
                'label':'Total water level (m)',
                'discretisation':{
                    'n_bins':7,
                    'strategy':'binned',
                    'bin_names':['VeryLow','Low','LowMid','Mid','MidHigh','High','VeryHigh'],
                    'bin_edges':np.arange(-1,3.0,0.5)
                },
                'child_nodes':[]
            },
            'TWL_less_Tide':{
                'label':'Total water level less tide (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL']
            },
            'MSL':{
                'label':'Mean sea level (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'Tide':{
                'label':'Tide (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL']
            }
        },
        'training_frac':0.8,
        'bootstrap_reps':1
    }
    ocean_model_dict = {
       'variables':{
           'Tm_offshore':{
                'label':'Wave period offshore (?)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'Hs_offshore':{
                'label':'Wave height offshore (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'Dir_offshore':{
                'label':'Wave direction offshore (degrees)',
                'discretisation':{
                    'n_bins':8,
                    'strategy':'kmeans',
                    'bin_names':['NNE','ENE','ESE','SSE','SSW','WSW','WNW','NNW']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'MSL':{
                'label':'Mean sea level (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'Tide':{
                'label':'Tide (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL']
            },
            'TWL':{
                'label':'Total water level (m)',
                'discretisation':{
                    'n_bins':7,
                    'strategy':'binned',
                    'bin_names':['VeryLow','Low','LowMid','Mid','MidHigh','High','VeryHigh'],
                    'bin_edges':np.arange(-1,3.0,0.5)
                },
                'child_nodes':[]
            },
            'TWL_less_Tide':{
                'label':'Total water level less tide (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL']
            },
            'reef_width':{
                'label':'Reef width (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'reef_depth':{
                'label':'Reef depth (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'forereef_slope':{
                'label':'Fore reef slope (degrees)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['TWL_less_Tide']
            },
            'shore_dir':{
                'label':'Shoreline direction (degrees)',
                'discretisation':{
                    'n_bins':3,
                    'strategy':'kmeans',
                    'bin_names':['NE','S','NW']
                },
                'child_nodes':['TWL_less_Tide']
            }
       },
        'training_frac':0.8,
        'bootstrap_reps':1
    }
    return(lagoon_model_dict,ocean_model_dict)

In [16]:
def location_probabilities(evidence_dict,model_dict,variable_list,df_profiles):
    '''
    
    Function for setting evidence and determing probabilties for twl at each point around the island based 
    on the reef characteristics at each location
    
    '''
    
    location_probabilities_dict = {}
    
    for index,row in df_profiles.iterrows():

        model_location_dict = model_location(model_dict,row,evidence_dict,variable_list)
        location_probabilities = model_location_dict['variables']['TWL']['resulting_probs'][0]

        df_location_probabilities = pd.DataFrame.from_dict(location_probabilities,orient='index')
        
        largest_cat = df_location_probabilities.idxmax()[0]

        location_probabilities_dict.update({
            (row.reef_long,row.reef_lat):\
                model_dict['variables']['TWL']['discretisation']['bin_names'].index(largest_cat)
        })
        
    return(location_probabilities_dict)

## Bootstrapping the data

In [6]:
# def test_figure(
#     view,tide_bin,wave_height_bin,wave_period_bin,wave_direction_bin,wind_u_bin,wind_v_bin,proj_time,time_min,time_max
#     ):
    
#     for lower_idx,upper_idx in zip(np.arange(time_min,len(inundation_dict['Time']),100),np.arange(100+time_min,len(inundation_dict['Time']),100)):
#         print(lower_idx,upper_idx)

    
#     # Extract the right model from the dictionary of model
#     lagoon_model_dict = model_dicts_through_time_dict[time_max]['lagoon']
#     ocean_model_dict = model_dicts_through_time_dict[time_min]['ocean']
    
#     ########### get the MSL bin based on slider value
#     msl_proj = SLR_proj_extractor(SL_proj_dict,'k14','26','{}'.format(proj_time))
#     bin_count = bin_locator(msl_proj,ocean_model_dict['variables']['MSL']['bin_edges'][0])
#     msl_bin = ocean_model_dict['variables']['MSL']['discretisation']['bin_names'][bin_count]
   
#     ################################################
    
#     # Create an empty dictionary for the evidence and populate as you go
#     ocean_evidence_dict = {}
    
#     for var_bin,var_name in zip([tide_bin,msl_bin,wave_height_bin,wave_period_bin,wave_direction_bin],
#                                 ['Tide','MSL','Hs_offshore','Tm_offshore','Dir_offshore']):

#         ## Set in the evidence dict to be as indicated in the dropdown
#         bin_index = ocean_model_dict['variables'][var_name]['discretisation']['bin_names'].index(var_bin)
#         # Create a list of the tide evidence (all zero except as indicated by dropdown. Dropdown=1)
#         evidence = [0 for x in ocean_model_dict['variables'][var_name]['discretisation']['bin_names']]
#         evidence[bin_index] = 1
#         ocean_evidence_dict.update({
#             var_name:evidence
#         })
    
#     # Create a list of variables that are location specific to set as evidence in the network
#     variable_list = ['reef_width','reef_depth','forereef_slope','shore_dir']
    
#     # get the probability dictionary
#     location_probabilities_dict = location_probabilities(ocean_evidence_dict,ocean_model_dict,variable_list,df_ocean_profiles)
    
#     # Create dataframe to plot
#     df_twl_locations = pd.DataFrame.from_dict(location_probabilities_dict,orient='index').rename(columns={0:'most_likely_twl'})
#     df_twl_locations['long'] = [long for long,lat in df_twl_locations.index]
#     df_twl_locations['lat'] = [lat for long,lat in df_twl_locations.index]
#     df_twl_locations.reset_index(drop=True,inplace=True)
    
#     #####################################################################
    
#     # Create an empty dictionary for the evidence and populate as you go
#     lagoon_evidence_dict = {}
    
#     for var_bin,var_name in zip([tide_bin,msl_bin,wave_height_bin,wave_period_bin,wave_direction_bin,wind_u_bin,wind_v_bin],
#                                 ['Tide','MSL','Hs_offshore','Tm_offshore','Dir_offshore','wind_u','wind_v']):

#         ## Set in the evidence dict to be as indicated in the dropdown
#         bin_index = lagoon_model_dict['variables'][var_name]['discretisation']['bin_names'].index(var_bin)
#         # Create a list of the tide evidence (all zero except as indicated by dropdown. Dropdown=1)
#         evidence = [0 for x in lagoon_model_dict['variables'][var_name]['discretisation']['bin_names']]
#         evidence[bin_index] = 1
#         lagoon_evidence_dict.update({
#             var_name:evidence
#         })
    
#     # Create a list of variables that are location specific to set as evidence in the network
#     variable_list = []
    
#     # get the probability dictionary
#     location_probabilities_dict = location_probabilities(lagoon_evidence_dict,lagoon_model_dict,variable_list,df_lagoon_profiles)
    
#     # Create dataframe to plot
#     df_twl_locations = pd.DataFrame.from_dict(location_probabilities_dict,orient='index').rename(columns={0:'most_likely_twl'})
#     df_twl_locations['long'] = [long for long,lat in df_twl_locations.index]
#     df_twl_locations['lat'] = [lat for long,lat in df_twl_locations.index]
#     df_twl_locations.reset_index(drop=True,inplace=True)

#     return()

# # compile the figure
# lagoon_model_dict,ocean_model_dict = initialise_model_dictionaries()
# tide_bins = ocean_model_dict['variables']['Tide']['discretisation']['bin_names']
# wave_height_bins = ocean_model_dict['variables']['Hs_offshore']['discretisation']['bin_names']
# wave_period_bin = ocean_model_dict['variables']['Tm_offshore']['discretisation']['bin_names']
# wave_direction_bin = ocean_model_dict['variables']['Dir_offshore']['discretisation']['bin_names']
# wind_u_bin = lagoon_model_dict['variables']['wind_u']['discretisation']['bin_names']
# wind_v_bin = lagoon_model_dict['variables']['wind_u']['discretisation']['bin_names']
# time = list(df_lagoon.time) 

# # Create the plot with the widget
# interact_manual(test_figure,
#                 view = widgets.Dropdown(options=['Map','Satellite'],value='Map',description='View type',disabled=False),
#                 tide_bin = widgets.Dropdown(options=tide_bins,value='Mid',description='Tide',disabled=False),
#                 wave_height_bin = widgets.Dropdown(options=wave_height_bins,value='Mid',description='Wave height',disabled=False),
#                 wave_period_bin = widgets.Dropdown(options=wave_period_bin,value='Mid',description='Wave period',disabled=False),
#                 wave_direction_bin = widgets.Dropdown(options=wave_direction_bin,value='NNE',description='Wave direction',disabled=False),
#                 wind_u_bin = widgets.Dropdown(options=wind_u_bin,value='Mid',description='Wind u',disabled=False),
#                 wind_v_bin = widgets.Dropdown(options=wind_v_bin,value='Mid',description='Wind v',disabled=False),
#                 proj_time = widgets.IntSlider(min=2020,max=2150,step=10,value=2020,description='SLR prediction'),
#                 time_min = widgets.Dropdown(options=time,value=time[0],description='Time',disabled=False),
#                 time_max = widgets.Dropdown(options=time,value=time[0],description='Time',disabled=False)
#                )



# One network per time step

In [7]:

model_dicts_through_time_dict = {}

time_min = inundation_dict['Time'][0]
time_step = timedelta(hours=24)
max_steps = 7

times_for_model = []

for step in np.arange(0,max_steps,1):
    time_at_step = time_min+time_step*step
    
    times_for_model.append(time_at_step)
    
times_for_model = times_for_model[:10]
    
for time_min,time_max in zip(times_for_model[:-1],times_for_model[1:]):
    lower_idx = np.abs([x-time_min for x in inundation_dict['Time']]).argmin()
    upper_idx = np.abs([x-time_max for x in inundation_dict['Time']]).argmin()
       
    inundation_time_slice_dict = {var:inundation_dict[var][lower_idx:upper_idx,:] for var in ['TWL','Tide']}
    inundation_time_slice_dict['Time'] = inundation_dict['Time'][lower_idx:upper_idx]
    inundation_time_slice_dict['Ptos'] = inundation_dict['Ptos']
    winds_time_slice_dict = {
        'wind_u':winds_dict['wind_u'][lower_idx:upper_idx],'wind_v':winds_dict['wind_v'][lower_idx:upper_idx]}
    waves_time_slice_dict = {var:waves_dict[var][lower_idx:upper_idx,:] for var in ['Diro','Hso','Tmo','Tpo']}
    waves_time_slice_dict['Timeo'] = waves_dict['Timeo'][lower_idx:upper_idx]
    
    tide_time_slice_dict = {'Tide':tide_dict['Tide'][lower_idx:upper_idx]}
    sla_time_slice_dict = {'MSL':sla_dict['MSL'][lower_idx:upper_idx]}
    time_slice_dict = {'time':time_dict['time'][lower_idx:upper_idx]}
    
    if time_min==times_for_model[0]:
        lagoon_model_dict,ocean_model_dict = initialise_model_dictionaries()
        predicted_lag_variables = []
    else:
        predicted_lag_variables = ['TWL_t_1']
        for var in predicted_lag_variables:
            df_ocean_profiles[var] = 0#[item for key,item in ocean_probabilities_dict.items()]
            
        ocean_model_dict['variables'].update({
                'TWL_t_1':{
                'label':'Total water level bin at t-1 (1-5?)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['1','2','3','4','5']
                },
                'child_nodes':['TWL']
            }
        })

    df_ocean,df_lagoon,ocean_data_dict,lagoon_data_dict = \
        preprocessing_points_spatially_temporally(df_lagoon_profiles,
                                                  df_ocean_profiles,
                                                  inundation_time_slice_dict,
                                                  winds_time_slice_dict,
                                                  waves_time_slice_dict,
                                                  tide_time_slice_dict,
                                                  sla_time_slice_dict,
                                                  time_slice_dict,
                                                  predicted_lag_variables)
    
    lagoon_model_dict,ocean_model_dict = create_BN_time_t(lagoon_model_dict,
                                                          lagoon_data_dict,
                                                          df_lagoon,
                                                          ocean_model_dict,
                                                          df_ocean,
                                                          ocean_data_dict)
    # Create an empty evidence dict
    evidence_dict = {}
    
    # Add evidence to model dict
    ocean_model_dict = BNModel().add_evidence_to_dict(ocean_model_dict,evidence_dict)
    lagoon_model_dict = BNModel().add_evidence_to_dict(lagoon_model_dict,evidence_dict)

    # Set evidence and get beliefs
    ocean_model_dict = BNModel().update_evidence(ocean_model_dict)
    lagoon_model_dict = BNModel().update_evidence(lagoon_model_dict)
    
    # Get the resulting TWL probabilities (right now for a generic location)
    resulting_probs_ocean = ocean_model_dict['variables']['TWL']['resulting_probs']
    resulting_probs_lagoon = lagoon_model_dict['variables']['TWL']['resulting_probs']
    
    model_dicts_through_time_dict.update({
        time_dict['time'][lower_idx]:{
            'lagoon':lagoon_model_dict,
            'ocean':ocean_model_dict
        }
    })

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_discretization.py:187: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn("Feature %d is constant and will be "
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_discretization.py:187: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn("Feature %d is constant and will be "
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_discretization.py:187: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn("Feature %d is constant and will be "
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_discretization.py:187: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn("Feature %d is constant and will be "
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_discretization.py:187: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn("Feature %d is constant

In [17]:
ocean_evidence_dict = {}
lagoon_evidence_dict = {}

# Create a list of variables that are location specific to set as evidence in the network
variable_list = ['reef_width','reef_depth','forereef_slope','shore_dir']

# get the probability dictionary
location_probabilities_dict = location_probabilities(ocean_evidence_dict,ocean_model_dict,variable_list,df_ocean_profiles)

# Create dataframe to plot
df_twl_locations = pd.gt.from_dict(location_probabilities_dict,orient='index').rename(columns={0:'most_likely_twl'})
df_twl_locations['long'] = [long for long,lat in df_twl_locations.index]
df_twl_locations['lat'] = [lat for long,lat in df_twl_locations.index]
df_twl_locations.reset_index(drop=True,inplace=True)

data_ocean = data2geojson(df_twl_locations)

NameError: name 'tide_bin' is not defined

In [15]:
ocean_model_dict

{'variables': {'Tm_offshore': {'label': 'Wave period offshore (?)',
   'discretisation': {'n_bins': 5,
    'strategy': 'kmeans',
    'bin_names': ['VeryLow', 'Low', 'Mid', 'High', 'VeryHigh']},
   'child_nodes': ['TWL_less_Tide'],
   'training_data_preprocessed': {0: array([8.64342022, 8.70792007, 8.90485859, ..., 1.55849063, 1.53075361,
           1.56545186])},
   'testing_data_preprocessed': {0: array([ 8.59606838,  8.79444218,  9.37001514,  9.85026932,  9.98466969,
           10.2612133 , 10.20583344,  8.75055695,  9.89011097, 10.25673866,
           10.2834692 , 10.29194069, 10.17378616,  8.83363056,  9.25190926,
            9.79903221,  9.89135361,  9.97934055,  8.92916203,  9.38593674,
            9.54708099,  9.98218536, 10.08782291, 10.14090157,  8.63446045,
            9.03453732, 10.22472668, 10.17245293, 10.11829376,  8.77729702,
            9.96271515, 10.0939703 , 10.3332653 ,  9.64385796,  9.96271515,
           10.22760105,  8.67266083,  8.88302326,  9.13686943, 10.2881

# Defining times of interest


In [8]:
# # Create an empty dictionary to have a look at how the resulting probabilties varied through time
# resulting_probs_dict = {}

# for time,model_dict['lagoon'] in model_dicts_through_time_dict.items():
    
#     #Create an empty dictionary of evidence for now
#     evidence_dict = {}
    
#     # Add evidence to model dict
#     model_location_dict = BNModel().add_evidence_to_dict(model_dict,evidence_dict)

#     # Set evidence and get beliefs
#     model_location_dict = BNModel().update_evidence(model_location_dict)
    
#     # Get the resulting TWL probabilities (right now for a generic location)
#     resulting_probs = model_location_dict['variables']['TWL']['resulting_probs']
    
#     # Add the resulting probabilities to a dictionary
#     resulting_probs_dict.update({
#         time:resulting_probs[0]
#     })

In [9]:
# resulting_probs_dict

# Interactive Plots

In [10]:
# # load in the reef and shoreline profile information
# df_ocean_profiles = pd.read_csv('/src/Dataset/D8_tarawa_inundation/Profiles_definition_outer_reef_xyxy_processed.txt')
# df_lagoon_profiles = pd.read_csv('/src/Dataset/D8_tarawa_inundation/Profiles_definition_inner_lagoon_xyxy.txt',delim_whitespace=True,header=None)
# df_lagoon_profiles.columns = ['reef_long','reef_lat','shore_long','shore_lat','reef_depth']


In [11]:
# def bin_locator(value,bin_edges):
#     '''
#     function used for determining the index of the appropriate bin for a numerical value.
#     '''
#     i=0
#     for edge_1,edge_2 in zip(bin_edges[:-1],bin_edges[1:]):
#         if (value>edge_1)&(value<=edge_2):
#             loc_bin = i
#         else:
#             i+=1
#             continue

#     if value<=bin_edges[0]:
#         loc_bin = 0

#     if value>=bin_edges[-1]:
#         loc_bin = len(bin_edges)-2

#     return(loc_bin)

# def model_location(model_dict,location_details,evidence_dict,variable_list):
    
#     '''
#     function for adding the location information for one side model to the evidence dictionary
#     '''
    
#     for variable in variable_list:

#         bin_edges = model_dict['variables'][variable]['bin_edges'][0]
#         value = location_details[variable]

#         var_bin = bin_locator(value,bin_edges)
        
#         evidence_array = [0]*(len(bin_edges)-1)
#         evidence_array[var_bin] = 1
        
#         evidence_dict.update({
#             variable:evidence_array
#         })

#     # Add evidence to model dict
#     model_location_dict = BNModel().add_evidence_to_dict(model_dict,evidence_dict)
    
#     # Set evidence and get beliefs
#     model_location_dict = BNModel().update_evidence(model_location_dict)
    
#     return(model_location_dict)

# def location_probabilities(evidence_dict,model_dict,variable_list,df_profiles):
#     '''
    
#     Function for setting evidence and determing probabilties for twl at each point around the island based 
#     on the reef characteristics at each location
    
#     '''
    
#     location_probabilities_dict = {}
# #     figure_dict = {}
    
#     for index,row in df_profiles.iterrows():

#         model_location_dict = model_location(model_dict,row,evidence_dict,variable_list)
#         location_probabilities = model_location_dict['variables']['TWL']['resulting_probs'][0]

#         df_location_probabilities = pd.DataFrame.from_dict(location_probabilities,orient='index')
        
# #         # Create figure for popup
# #         fig = plt.figure(figsize=(2,2))
# #         plt.bar(x=df_location_probabilities.index,height=df_location_probabilities[0])
# #         plt.savefig('{}_{}.png'.format(int(row.reef_long*1000),int(row.reef_lat*1000)))
# #         plt.close()
        
#         largest_cat = df_location_probabilities.idxmax()[0]

#         location_probabilities_dict.update({
#             (row.reef_long,row.reef_lat):\
#                 model_dict['variables']['TWL']['discretisation']['bin_names'].index(largest_cat)
#         })
        
# #         figure_dict.update({
# #             (row.reef_long,row.reef_lat):fig#html_graph
# #         })
        
#     return(location_probabilities_dict)

# def data2geojson(df):
#     features = []
#     insert_features = lambda X: features.append(
#             geojson.Feature(geometry=geojson.Point((X["long"],
#                                                     X["lat"])),
#                             properties=dict(name=X["most_likely_twl"])))
#     df.apply(insert_features, axis=1)
        
#     return(geojson.FeatureCollection(features))

# # Load SLR Projections
# data_location = "/src/Dataset/D7_MSL_projections/"
# file_name = "distributions_dict"
# with open("{}{}.json".format(data_location,file_name), 'r') as fp:
#     SL_proj_dict = json.load(fp)
    
# def SLR_proj_extractor(SL_proj_dict,AIS_config,rcp,year):
#     '''
#     Function for getting SLR projections for a given Antarctic icesheet, rcp and year
#     Years start as 2020 and go up in lots of 10 until 2150 (2100 for dp16)
#     '''
#     SLR_prob_dict = SL_proj_dict["('{}', '{}', {})".format(AIS_config,rcp,year)]
#     SLR_median_prob = np.max([float(x) for x in list(SLR_prob_dict.keys())])
#     SLR_median_MSL = float(SLR_prob_dict[str(SLR_median_prob)])/1000 #units is m

#     return(SLR_median_MSL)

In [12]:
# def test_figure(
#     view,tide_bin,wave_height_bin,wave_period_bin,wave_direction_bin,wind_u_bin,wind_v_bin,proj_time,time
#     ):
#     # Extract the right model from the dictionary of model
#     lagoon_model_dict = model_dicts_through_time_dict[time]['lagoon']
#     ocean_model_dict = model_dicts_through_time_dict[time]['ocean']
    
#     ########### get the MSL bin based on slider value
#     msl_proj = SLR_proj_extractor(SL_proj_dict,'k14','26','{}'.format(proj_time))
# #     ocean_model_dict['variables']['MSL']['bin_edges'][0]
#     bin_count = bin_locator(msl_proj,ocean_model_dict['variables']['MSL']['bin_edges'][0])
#     msl_bin = ocean_model_dict['variables']['MSL']['discretisation']['bin_names'][bin_count]
    
#     if view == 'Map':
#         map_osm = folium.Map(location=[1.448888, 172.991794],zoom_start=11)
#     elif view == 'Satellite':
#         token = "pk.eyJ1Ijoic2hhbm5vbi1iZW5ndHNvbiIsImEiOiJja3F1Y2Q0dHEwMzYwMm9wYmtzYzk2bDZuIn0.5jGMyEiJdmXs1HL7x3ThPw" # your mapbox token
#         tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)

#         map_osm = folium.Map(location=[1.448888, 172.991794], zoom_start=11, tiles=tileurl, attr='Mapbox')
        
#     twl_bin_edges = [round(x,2) for x in ocean_model_dict['variables']['TWL']['bin_edges'][0]]
#     twl_bins = ocean_model_dict['variables']['TWL']['discretisation']['bin_names']

#     colours_rgb = matplotlib.pyplot.get_cmap('seismic')(np.arange(0,1+1/len(twl_bins),1/(len(twl_bins)-1)))
#     colour_hex_dict = {i:rgb2hex(int(255*colours_rgb[i][0]),int(255*colours_rgb[i][1]),int(255*(colours_rgb[i][2]))) for i in np.arange(0,len(twl_bins),1)}
    
#     ################################################
    
#     # Create an empty dictionary for the evidence and populate as you go
#     ocean_evidence_dict = {}
    
#     for var_bin,var_name in zip([tide_bin,msl_bin,wave_height_bin,wave_period_bin,wave_direction_bin],
#                                 ['Tide','MSL','Hs_offshore','Tm_offshore','Dir_offshore']):

#         ## Set in the evidence dict to be as indicated in the dropdown
#         bin_index = ocean_model_dict['variables'][var_name]['discretisation']['bin_names'].index(var_bin)
#         # Create a list of the tide evidence (all zero except as indicated by dropdown. Dropdown=1)
#         evidence = [0 for x in ocean_model_dict['variables'][var_name]['discretisation']['bin_names']]
#         evidence[bin_index] = 1
#         ocean_evidence_dict.update({
#             var_name:evidence
#         })
    
#     # Create a list of variables that are location specific to set as evidence in the network
#     variable_list = ['reef_width','reef_depth','forereef_slope','shore_dir']
    
#     # get the probability dictionary
#     location_probabilities_dict = location_probabilities(ocean_evidence_dict,ocean_model_dict,variable_list,df_ocean_profiles)
    
#     # Create dataframe to plot
#     df_twl_locations = pd.DataFrame.from_dict(location_probabilities_dict,orient='index').rename(columns={0:'most_likely_twl'})
#     df_twl_locations['long'] = [long for long,lat in df_twl_locations.index]
#     df_twl_locations['lat'] = [lat for long,lat in df_twl_locations.index]
#     df_twl_locations.reset_index(drop=True,inplace=True)
    
#     data_ocean = data2geojson(df_twl_locations)
    
#     colors_hex_points_ocean = [colour_hex_dict[x] for x in df_twl_locations.most_likely_twl]
    
#     #####################################################################
    
#     # Create an empty dictionary for the evidence and populate as you go
#     lagoon_evidence_dict = {}
    
#     for var_bin,var_name in zip([tide_bin,msl_bin,wave_height_bin,wave_period_bin,wave_direction_bin,wind_u_bin,wind_v_bin],
#                                 ['Tide','MSL','Hs_offshore','Tm_offshore','Dir_offshore','wind_u','wind_v']):

#         ## Set in the evidence dict to be as indicated in the dropdown
#         bin_index = lagoon_model_dict['variables'][var_name]['discretisation']['bin_names'].index(var_bin)
#         # Create a list of the tide evidence (all zero except as indicated by dropdown. Dropdown=1)
#         evidence = [0 for x in lagoon_model_dict['variables'][var_name]['discretisation']['bin_names']]
#         evidence[bin_index] = 1
#         lagoon_evidence_dict.update({
#             var_name:evidence
#         })
    
#     # Create a list of variables that are location specific to set as evidence in the network
#     variable_list = []
    
#     # get the probability dictionary
#     location_probabilities_dict = location_probabilities(lagoon_evidence_dict,lagoon_model_dict,variable_list,df_lagoon_profiles)
    
#     # Create dataframe to plot
#     df_twl_locations = pd.DataFrame.from_dict(location_probabilities_dict,orient='index').rename(columns={0:'most_likely_twl'})
#     df_twl_locations['long'] = [long for long,lat in df_twl_locations.index]
#     df_twl_locations['lat'] = [lat for long,lat in df_twl_locations.index]
#     df_twl_locations.reset_index(drop=True,inplace=True)
    
#     data_lagoon = data2geojson(df_twl_locations)
    
#     colors_hex_points_lagoon = [colour_hex_dict[x] for x in df_twl_locations.most_likely_twl]
    
#     #####################################################################
    
#     features_list = data_ocean['features']+data_lagoon['features']
    
#     data = data_ocean
#     data.update({
#         'features':features_list
#     })
    
#     colors_hex_points = colors_hex_points_ocean+colors_hex_points_lagoon
    
#     #####################################################################

#     for feature,color in zip(features_list,colors_hex_points):
#         feature['properties'] = {'color':color,'weight':1,'markerColor':color,'fillOpacity':1,'fillColor':color}
#         long,lat = feature['geometry']['coordinates']
        
#         marker = folium.CircleMarker([lat,long],color=color,
#                                     # popup='<img src={}_{}.png>'.format(int(long*1000),int(lat*1000)),
#                                    fill_color=color,fill=True,fill_opacity='1',radius=5)
#         marker.add_to(map_osm)
        
#     twl_bin_edge_labels = ['{} to {} m'.format(
#         x,y) for x,y in zip(twl_bin_edges[:-1],twl_bin_edges[1:])]
        
#     output_list = []
#     for rgb_color in colours_rgb:
#         output = plt.scatter([],[],color=rgb_color)
#         output_list.append(output)
        
#     legend = plt.legend(output_list,twl_bin_edge_labels,title='Total water level anomaly',fontsize=10)
#     plt.setp(legend.get_title(),fontsize=12)
    
#     plt.axis('off')
#     plt.savefig('legend.png')
    
#     plt.close()
    
#     url = (
#         "legend.png"
#     )    
    
#     FloatImage(url, bottom=55, left=55).add_to(map_osm)
    
#     map_osm.save('test.html')
        
#     return(map_osm)

# # compile the figure
# lagoon_model_dict,ocean_model_dict = initialise_model_dictionaries()
# tide_bins = ocean_model_dict['variables']['Tide']['discretisation']['bin_names']
# wave_height_bins = ocean_model_dict['variables']['Hs_offshore']['discretisation']['bin_names']
# wave_period_bin = ocean_model_dict['variables']['Tm_offshore']['discretisation']['bin_names']
# wave_direction_bin = ocean_model_dict['variables']['Dir_offshore']['discretisation']['bin_names']
# wind_u_bin = lagoon_model_dict['variables']['wind_u']['discretisation']['bin_names']
# wind_v_bin = lagoon_model_dict['variables']['wind_u']['discretisation']['bin_names']
# time = list(model_dicts_through_time_dict.keys())  

# # Create the plot with the widget
# map_osm = interact_manual(test_figure,
#                 view = widgets.Dropdown(options=['Map','Satellite'],value='Map',description='View type',disabled=False),
#                 tide_bin = widgets.Dropdown(options=tide_bins,value='Mid',description='Tide',disabled=False),
#                 wave_height_bin = widgets.Dropdown(options=wave_height_bins,value='Mid',description='Wave height',disabled=False),
#                 wave_period_bin = widgets.Dropdown(options=wave_period_bin,value='Mid',description='Wave period',disabled=False),
#                 wave_direction_bin = widgets.Dropdown(options=wave_direction_bin,value='NNE',description='Wave direction',disabled=False),
#                 wind_u_bin = widgets.Dropdown(options=wind_u_bin,value='Mid',description='Wind u',disabled=False),
#                 wind_v_bin = widgets.Dropdown(options=wind_v_bin,value='Mid',description='Wind v',disabled=False),
#                 proj_time = widgets.IntSlider(min=2020,max=2150,step=10,value=2020,description='SLR prediction'),
#                 time = widgets.Dropdown(options=time,value=727930.0,description='Time',disabled=False)
#                )

# map_osm